# Benchmark of existing approaches for detecting machine-generated text

## Contents 
1. [Solaiman](#Solaiman)
    1. [Install dependencies](#Install-dependencies)
    1. [Solaiman Code](#Solaiman-Code)
2. [Data Exploration](#Data-exploration)


## Solaiman

_Irene Solaiman, Miles Brundage, Jack Clark, Amanda Askell, Ariel Herbert-Voss, Jeff Wu, Alec Radford,
Gretchen Krueger, Jong Wook Kim, Sarah Kreps, Miles McCain, Alex Newhouse, Jason Blazakis, Kris McGuffie, and Jasmine Wang. 2019. Release strategies and the social impacts of language models._

### Install dependencies
Dependencies from: [https://github.com/HendrikStrobelt/detecting-fake-text/blob/master/requirements.txt](detecting-fake-text/requirements.txt).

### Solaiman Code
Source code: [https://github.com/openai/gpt-2-output-dataset/tree/master/detector](https://github.com/openai/gpt-2-output-dataset/tree/master/detector)

Logic extracted form `server.py`

In [1]:
import transformers
assert transformers.__version__ == '2.9.1', "Use transformers 2.9.1. It is available in the conda environment transf291"

/home/matteonardelli/miniconda3/envs/transf291/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer
import json
import torch
from urllib.parse import urlparse, unquote

#### Select model to use RoBERTa base or large

In [3]:
model_name = 'roberta-large'
#model_name = 'roberta-base'

#### Define model, tokenizer, and basic functions

In [4]:
model = RobertaForSequenceClassification.from_pretrained(model_name)
tokenizer = RobertaTokenizer.from_pretrained(model_name)
device='cuda' if torch.cuda.is_available() else 'cpu'

device = 'cpu'

In [5]:
def evaluate(query):
    tokens = tokenizer.encode(query)
    all_tokens = len(tokens)
    tokens = tokens[:tokenizer.max_len - 2]
    used_tokens = len(tokens)
    tokens = torch.tensor([tokenizer.bos_token_id] + tokens + [tokenizer.eos_token_id]).unsqueeze(0)
    mask = torch.ones_like(tokens)

    with torch.no_grad():
        logits = model(tokens.to(device), attention_mask=mask.to(device))[0]
        probs = logits.softmax(dim=-1)

    fake, real = probs.detach().cpu().flatten().numpy().tolist()

    # Original: 
#    return json.dumps(dict(
#         all_tokens=all_tokens,
#         used_tokens=used_tokens,
#         real_probability=real,
#         fake_probability=fake
#     ))

# Changed to return only the binary classification result. 1 if sentence is likely machine-generated.
    return (fake > 0.5, fake, real)

def initialize(checkpoint):
#     if checkpoint.startswith('gs://'):
#         print(f'Downloading {checkpoint}', file=sys.stderr)
#         subprocess.check_output(['gsutil', 'cp', checkpoint, '.'])
#         checkpoint = os.path.basename(checkpoint)
#         assert os.path.isfile(checkpoint)

    print(f'Loading checkpoint from {checkpoint}')
    data = torch.load(checkpoint, map_location='cpu')
    model.load_state_dict(data['model_state_dict'])
    model.eval()

#### Download finetuned model and load it into RoBERTa

In [6]:
# !wget https://openaipublic.azureedge.net/gpt-2/detector-models/v1/detector-base.pt
# !wget https://openaipublic.azureedge.net/gpt-2/detector-models/v1/detector-large.pt

In [7]:
initialize('detector-large.pt')
# initialize('detector-base.pt')

Loading checkpoint from detector-large.pt


In [8]:
evaluate("hello world")

(True, 0.702642023563385, 0.297357976436615)

## Data exploration

In [9]:
import json
import pandas as pd
import os
import time

In [10]:
data_path = "./data"
datasets = sorted([f for f in os.listdir(data_path) if os.path.isfile(os.path.join(data_path, f))])

In [11]:
datasets

['.DS_Store',
 '175b_samples.jsonl',
 'large-762M-k40.test.jsonl',
 'large-762M-k40.train.jsonl',
 'large-762M-k40.valid.jsonl',
 'large-762M.test.jsonl',
 'large-762M.train.jsonl',
 'large-762M.valid.jsonl',
 'medium-345M-k40.test.jsonl',
 'medium-345M-k40.train.jsonl',
 'medium-345M-k40.valid.jsonl',
 'medium-345M.test.jsonl',
 'medium-345M.train.jsonl',
 'medium-345M.valid.jsonl',
 'small-117M-k40.test.jsonl',
 'small-117M-k40.train.jsonl',
 'small-117M-k40.valid.jsonl',
 'small-117M.test.jsonl',
 'small-117M.train.jsonl',
 'small-117M.valid.jsonl',
 'webtext.test.jsonl',
 'webtext.train.jsonl',
 'webtext.train.jsonl.clean100k.csv',
 'webtext.valid.jsonl',
 'xl-1542M-k40.test.jsonl',
 'xl-1542M-k40.train.jsonl',
 'xl-1542M-k40.valid.jsonl',
 'xl-1542M.test.jsonl',
 'xl-1542M.train.jsonl',
 'xl-1542M.train.jsonl.clean100k.csv',
 'xl-1542M.valid.jsonl']

### Human-written data

In [14]:
# ds_hw_filename = 'webtext.train.jsonl'
# ds_hw = pd.read_json(os.path.join(data_path, ds_hw_filename), lines = True)     

ds_hw_filename = 'webtext.train.jsonl.clean100k.csv'
ds_hw = pd.read_csv(os.path.join(data_path, ds_hw_filename))  

# with open(os.path.join(data_path, ds_hw_filename), 'r') as file:
#    ds_hw = [x for x in file.readlines() if x is not None and x != "\n"]     

In [15]:
ds_hw

,id,ended,length,text,english words,english %,non-eng vocabs
0,172092,True,634,Everton goalkeeper Tim Howard has signed a con...,488,95,7
1,208655,True,264,That addresses performance. As for battery lif...,202,97,1
2,25537,False,1024,"Posted on February 18, 2014 by Bryan Ball\n\nB...",819,97,6
3,81025,True,335,Jasper Jolly\n\nhe US dollar has reached its h...,266,97,5
4,6966,True,48,I received a package containing food contents ...,41,100,0
...,...,...,...,...,...,...,...
99995,22089,False,1024,Who Pays\n\nAmbulance Billing\n\nThe Ministry ...,785,100,0
99996,146668,True,466,Description\n\nAutomatic Bottle Opener\n\n\nTh...,366,98,5
99997,100597,True,89,"We have created a bunch of 'how-to' videos, to...",66,95,2
99998,64251,True,72,$14.99\n\nEasily & securely mount your iPhone ...,42,95,2


In [16]:
start = time.time()
payload = evaluate(ds_hw.iloc[20].text)
# payload = evaluate(ds_hw[20])
end = time.time()
print("{:.2f} Seconds for a check with GPT-2".format(end - start))
print(f"machine generated? {payload}")

0.19 Seconds for a check with GPT-2
machine generated? (False, 0.0009526731446385384, 0.9990473389625549)


### Machine-generated data

In [17]:
# ds_mg_filename = 'small-117M.train.jsonl'
# ds_mg = pd.read_json(os.path.join(data_path, ds_mg_filename), lines = True)  
ds_mg_filename = 'xl-1542M.train.jsonl.clean100k.csv'
ds_mg = pd.read_csv(os.path.join(data_path, ds_mg_filename))  
# 
# with open(os.path.join(data_path, ds_mg_filename), 'r') as file:
#    ds_mg = [x for x in file.readlines() if x is not None and x != "\n"]

In [18]:
ds_mg

,id,text,length,ended,english words,english %,non-eng vocabs
0,140132,Dr Dollemayne said he was ever mindful of the ...,682,True,529,97,8
1,43198,James Raymond is among the authors whose artic...,1024,False,715,95,35
2,58415,ZANSKY PORTSMOUTH in Belfast admitted download...,747,True,582,96,18
3,223826,A relief that has become the byword for writer...,1024,False,789,98,14
4,206739,$78.00 / Sold Out\n\n(SOLD OUT)\n\nCompared to...,504,True,355,95,11
...,...,...,...,...,...,...,...
99995,59284,Install with only 1 click! Removes the first p...,39,True,28,96,1
99996,80292,Riun Davis named the starting quarterback of t...,884,True,681,97,14
99997,199811,Amtrak train derailed in Oklahoma after curtai...,711,True,571,98,6
99998,19150,"Gary Caradori, The Associated Press\n\n\nCARAC...",245,True,179,95,7


In [19]:
start = time.time()
payload = evaluate(ds_mg.iloc[20].text)
# payload = evaluate(ds_mg[20])
end = time.time()
print("{:.2f} Seconds for a check with GPT-2".format(end - start))
print(f"machine generated? {payload}")

Token indices sequence length is longer than the specified maximum sequence length for this model (1024 > 512). Running this sequence through the model will result in indexing errors


0.53 Seconds for a check with GPT-2
machine generated? (True, 0.9997244477272034, 0.0002755813766270876)


## Evaluation
### Auxiliary functions

In [20]:
import numpy as np
from sklearn.utils import shuffle

def create_dataset(raw_data, is_machine_generated=True):
    X = np.array(raw_data)
    if is_machine_generated:
        y = np.ones_like(X, dtype=int)
    else:
        y = np.zeros_like(X, dtype=int)
    return pd.DataFrame(data={'X':X, 'y':y})

def concat_and_shuffle(datasets):
    return shuffle(pd.concat(datasets)).reset_index()

In [21]:
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

def safe_macro_f1(y, y_pred):
    """
    Macro-averaged F1, forcing `sklearn` to report as a multiclass
    problem even when there are just two classes. `y` is the list of
    gold labels and `y_pred` is the list of predicted labels.

    """
    return f1_score(y, y_pred, average='macro', pos_label=None)

def safe_accuracy(y, y_pred):
    return accuracy_score(y, y_pred, normalize=True)

def gtc_evaluate(
        dataset,
        model,
        score_func=safe_accuracy):

    # Predictions if we have labels:
    y_ds, preds = model.predict(dataset)
    confusion = confusion_matrix(y_ds, preds)
    score = score_func(y_ds, preds)
        
    # Return the overall scores and other experimental info:
    return {
        'model': model,
#        'labeled_outputs': y_ds, 
#        'predictions': preds, 
        'confusion_matrix': confusion, 
        'score': score }


In [22]:
class Wrapper:
    def __init__(self):
        pass

    def predict(self, dataset, truncateAt=512):
        y_pred = []
        y_dataset = []
        
        totalsize = len(dataset)
        progressupdate = max(min(100, int(totalsize/100)), 1)
        prog = 0
        print(f"progupd: {progressupdate}; m(m(100, {int(totalsize/100)}), 1)")
        for index, row in dataset.iterrows():
            prog+=1
            if not row['X']: 
                print("Skipping sentence because it appears to be empty")
                continue
            try:
                truncated_sentence = row['X'][:truncateAt] if len(row['X']) > truncateAt else row['X']
                (is_generated, fake, real) = evaluate(truncated_sentence)
                y_pred.append(1 if is_generated else 0)     
                y_dataset.append(row['y'])
                
                if (index) % progressupdate == 0:
                    print(f"... progress: {index} ({prog / totalsize})")
            except:
                print(f"Error while processing the query: {row['X']}")
                break
        return (y_dataset, y_pred)

In [23]:
solaiman_detector = Wrapper()

### Playing around

In [24]:
# mg_pd = create_dataset(ds_mg["text"], is_machine_generated=True)
# hw_pd = create_dataset(ds_hw["text"], is_machine_generated=False)
# shuffled = concat_and_shuffle([mg_pd, hw_pd])
# sample_df = shuffled.sample(n=10, random_state=1)

# creating dataset from clean100k csv files
mg_pd = create_dataset(ds_mg["text"], is_machine_generated=True)
hw_pd = create_dataset(ds_hw["text"], is_machine_generated=False)

In [28]:
len(mg_pd)

100000

Testing whether the `gtc_evaluate()` works as expected. By default, we use accuracy to measure the score.

In [26]:
def gtc_evaluate_with_checkpoints(dataset, 
        model,
        checkpoint = 10000,
        score_func=safe_accuracy):
    length = len(dataset)
    splits = min(int(length/checkpoint), length) 
    confmatrix_sum = np.zeros((2, 2))
    for i in range(splits + 1):
        dataset_split = dataset[i * checkpoint : (i+1)*checkpoint]
        if dataset_split.empty:
            break
        partial_r = gtc_evaluate(dataset_split, solaiman_detector, score_func=score_func)
        print (f"----- split #{i} (size: {len(dataset_split)}) -----\n {partial_r['confusion_matrix']}\n----- acc: {partial_r['score']} -----")
        confmatrix_sum += partial_r['confusion_matrix']
    print(f"====================================")
    print(f" Final confusion matrix:\n {confmatrix_sum}")
    (tn, fp, fn, tp) = confmatrix_sum.ravel()
    accuracy = (tp +tn) / (tp + tn + fp + fn)
    print(f" Final accuracy: {accuracy}")
    print(f"====================================")

In [29]:
%%time
gtc_evaluate_with_checkpoints(mg_pd, solaiman_detector, checkpoint=10000, score_func=safe_accuracy)

progupd: 100; m(m(100, 100), 1)
... progress: 0 (0.0001)
... progress: 100 (0.0101)
... progress: 200 (0.0201)
... progress: 300 (0.0301)
... progress: 400 (0.0401)
... progress: 500 (0.0501)
... progress: 600 (0.0601)
... progress: 700 (0.0701)
... progress: 800 (0.0801)
... progress: 900 (0.0901)
... progress: 1000 (0.1001)
... progress: 1100 (0.1101)
... progress: 1200 (0.1201)
... progress: 1300 (0.1301)
... progress: 1400 (0.1401)
... progress: 1500 (0.1501)
... progress: 1600 (0.1601)
... progress: 1700 (0.1701)
... progress: 1800 (0.1801)
... progress: 1900 (0.1901)
... progress: 2000 (0.2001)
... progress: 2100 (0.2101)
... progress: 2200 (0.2201)
... progress: 2300 (0.2301)
... progress: 2400 (0.2401)
... progress: 2500 (0.2501)
... progress: 2600 (0.2601)
... progress: 2700 (0.2701)
... progress: 2800 (0.2801)
... progress: 2900 (0.2901)
... progress: 3000 (0.3001)
... progress: 3100 (0.3101)
... progress: 3200 (0.3201)
... progress: 3300 (0.3301)
... progress: 3400 (0.3401)


... progress: 27800 (0.7801)
... progress: 27900 (0.7901)
... progress: 28000 (0.8001)
... progress: 28100 (0.8101)
... progress: 28200 (0.8201)
... progress: 28300 (0.8301)
... progress: 28400 (0.8401)
... progress: 28500 (0.8501)
... progress: 28600 (0.8601)
... progress: 28700 (0.8701)
... progress: 28800 (0.8801)
... progress: 28900 (0.8901)
... progress: 29000 (0.9001)
... progress: 29100 (0.9101)
... progress: 29200 (0.9201)
... progress: 29300 (0.9301)
... progress: 29400 (0.9401)
... progress: 29500 (0.9501)
... progress: 29600 (0.9601)
... progress: 29700 (0.9701)
... progress: 29800 (0.9801)
... progress: 29900 (0.9901)
----- split #2 (size: 10000) -----
 [[   0    0]
 [1099 8901]]
----- acc: 0.8901 -----
progupd: 100; m(m(100, 100), 1)
... progress: 30000 (0.0001)
... progress: 30100 (0.0101)
... progress: 30200 (0.0201)
... progress: 30300 (0.0301)
... progress: 30400 (0.0401)
... progress: 30500 (0.0501)
... progress: 30600 (0.0601)
... progress: 30700 (0.0701)
... progres

... progress: 54900 (0.4901)
... progress: 55000 (0.5001)
... progress: 55100 (0.5101)
... progress: 55200 (0.5201)
... progress: 55300 (0.5301)
... progress: 55400 (0.5401)
... progress: 55500 (0.5501)
... progress: 55600 (0.5601)
... progress: 55700 (0.5701)
... progress: 55800 (0.5801)
... progress: 55900 (0.5901)
... progress: 56000 (0.6001)
... progress: 56100 (0.6101)
... progress: 56200 (0.6201)
... progress: 56300 (0.6301)
... progress: 56400 (0.6401)
... progress: 56500 (0.6501)
... progress: 56600 (0.6601)
... progress: 56700 (0.6701)
... progress: 56800 (0.6801)
... progress: 56900 (0.6901)
... progress: 57000 (0.7001)
... progress: 57100 (0.7101)
... progress: 57200 (0.7201)
... progress: 57300 (0.7301)
... progress: 57400 (0.7401)
... progress: 57500 (0.7501)
... progress: 57600 (0.7601)
... progress: 57700 (0.7701)
... progress: 57800 (0.7801)
... progress: 57900 (0.7901)
... progress: 58000 (0.8001)
... progress: 58100 (0.8101)
... progress: 58200 (0.8201)
... progress: 

... progress: 82000 (0.2001)
... progress: 82100 (0.2101)
... progress: 82200 (0.2201)
... progress: 82300 (0.2301)
... progress: 82400 (0.2401)
... progress: 82500 (0.2501)
... progress: 82600 (0.2601)
... progress: 82700 (0.2701)
... progress: 82800 (0.2801)
... progress: 82900 (0.2901)
... progress: 83000 (0.3001)
... progress: 83100 (0.3101)
... progress: 83200 (0.3201)
... progress: 83300 (0.3301)
... progress: 83400 (0.3401)
... progress: 83500 (0.3501)
... progress: 83600 (0.3601)
... progress: 83700 (0.3701)
... progress: 83800 (0.3801)
... progress: 83900 (0.3901)
... progress: 84000 (0.4001)
... progress: 84100 (0.4101)
... progress: 84200 (0.4201)
... progress: 84300 (0.4301)
... progress: 84400 (0.4401)
... progress: 84500 (0.4501)
... progress: 84600 (0.4601)
... progress: 84700 (0.4701)
... progress: 84800 (0.4801)
... progress: 84900 (0.4901)
... progress: 85000 (0.5001)
... progress: 85100 (0.5101)
... progress: 85200 (0.5201)
... progress: 85300 (0.5301)
... progress: 

In [ ]:
# %%time
# gtc_evaluate(mg_pd[:10], solaiman_detector, score_func=safe_accuracy)

The Solaiman's model warns that sentences should have less than 512 tokens. So, let's try by filtering out senteces with more tokens than that.

### GPT-3 Machine generated text

In [57]:
gpt3_data_path = "./../../get_text_detect_space/datasets/GPT3"
gpt3_ds_mg_filename = '175b_samples.jsonl'
with open(os.path.join(gpt3_data_path, gpt3_ds_mg_filename), 'r') as file:
    gpt3_ds_mg = [x for x in file.readlines() if x is not None and x != "\n"]

In [58]:
gpt3_mg_pd = create_dataset(gpt3_ds_mg, is_machine_generated=True)

In [59]:
gpt3_mg_pd

,X,y
0,"""Glacier Ridge Christian School\n\nGlacier Rid...",1
1,"""His father was a professor and his mother was...",1
2,"""Image caption Bailiffs removed protesters fro...",1
3,"""Contents\n\n1. The Meaning of Love\n\n2. Our ...",1
4,"""Interviews\n\nInterview with Ariel Efraim (So...",1
...,...,...
480,"""Get Rangers updates directly to your inbox Su...",1
481,"""Posted 01 September 2010 - 07:51 PM\n\nUsing ...",1
482,"""Jeff Chapman\n\nJeffrey Scott Chapman (born M...",1
483,"""You can look into the Issue Tracker and repor...",1


In [61]:
%%time
gtc_evaluate_with_checkpoints(gpt3_mg_pd, solaiman_detector, checkpoint=1000, score_func=safe_accuracy)

progupd: 4; m(m(100, 4), 1)
... progress: 0 (0.002061855670103093)
... progress: 4 (0.010309278350515464)
... progress: 8 (0.018556701030927835)
... progress: 12 (0.026804123711340205)
... progress: 16 (0.03505154639175258)
... progress: 20 (0.04329896907216495)
... progress: 24 (0.05154639175257732)
... progress: 28 (0.05979381443298969)
... progress: 32 (0.06804123711340206)
... progress: 36 (0.07628865979381444)
... progress: 40 (0.08453608247422681)
... progress: 44 (0.09278350515463918)
... progress: 48 (0.10103092783505155)
... progress: 52 (0.10927835051546392)
... progress: 56 (0.11752577319587629)
... progress: 60 (0.12577319587628866)
... progress: 64 (0.13402061855670103)
... progress: 68 (0.1422680412371134)
... progress: 72 (0.15051546391752577)
... progress: 76 (0.15876288659793814)
... progress: 80 (0.1670103092783505)
... progress: 84 (0.17525773195876287)
... progress: 88 (0.18350515463917524)
... progress: 92 (0.19175257731958764)
... progress: 96 (0.2)
... progress: 

Token indices sequence length is longer than the specified maximum sequence length for this model (1026 > 512). Running this sequence through the model will result in indexing errors


... progress: 188 (0.38969072164948454)
... progress: 192 (0.3979381443298969)
... progress: 196 (0.4061855670103093)
... progress: 200 (0.4144329896907217)
... progress: 204 (0.422680412371134)
... progress: 208 (0.4309278350515464)
... progress: 212 (0.43917525773195876)
... progress: 216 (0.44742268041237115)
... progress: 220 (0.4556701030927835)
... progress: 224 (0.4639175257731959)
... progress: 228 (0.47216494845360824)
... progress: 232 (0.48041237113402063)
... progress: 236 (0.488659793814433)
... progress: 240 (0.49690721649484537)
... progress: 244 (0.5051546391752577)
... progress: 248 (0.51340206185567)
... progress: 252 (0.5216494845360825)
... progress: 256 (0.5298969072164949)
... progress: 260 (0.5381443298969072)
... progress: 264 (0.5463917525773195)
... progress: 268 (0.554639175257732)
... progress: 272 (0.5628865979381443)
... progress: 276 (0.5711340206185567)
... progress: 280 (0.5793814432989691)
... progress: 284 (0.5876288659793815)
... progress: 288 (0.595

# TODO


- [x] do not combine fake and real; treat dataset independently
- [ ] verificare che non generi errori su termini non noti
    - [x] errore se stringhe vuote; risolto
    - [x] accettare solo stringhe di lungh < 512
- [x] truncate sentences (less then 512)
- [x] evaluate using checkpoints
- [ ] effettua performance su dataset clean100k
- [ ] scarica dataset grover (25k per model, mixed fake/real)
- [ ] scarica il dataset GPT-3 generated samples (485 samples)
- [ ] stampare progressi periodici 